In [2]:
# Import pandas
import pandas as pd

In [3]:
# CSV file to pandas dataframe
CSVFile = 'CA_KERN_Auctions.csv'
auctions = pd.read_csv(CSVFile)

In [4]:
auctions.head()

,Auction ID,Unique #,Parcel Number,State,County Name,Parcel Location,Legal 1,Legal 2,Legal 3,Legal 4,...,Address 5,Address 6,Prior Lienholder 1,Prior Lien Year 1,Prior Lienholder 2,Prior Lien Year 2,Minimum Bid Owed,Max Overbid,Legal Text,Property Description
0,3065,3,002-072-06-00-7,CA,Kern,3025 Q ST BAKERSFIELD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,34400.0,NaN,NaN,Tax Rate Area: 001-001Use Code: 103Zoning Clas...
1,3066,4,002-280-27-00-5,CA,Kern,1300 33RD ST BAKERSFIELD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13200.0,NaN,NaN,Tax Rate Area: 001-495Use Code: 3100Zoning Cla...
2,3067,5,005-342-08-00-2,CA,Kern,1303 26TH ST BAKERSFIELD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,37200.0,NaN,NaN,Tax Rate Area: 001-495Use Code: 3010Zoning Cla...
3,3070,8,007-192-05-00-4,CA,Kern,1921 ORANGE ST BAKERSFIELD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11500.0,NaN,NaN,Tax Rate Area: 001-001Use Code: 103Zoning Clas...
4,3071,9,008-213-03-00-7,CA,Kern,137 MYRTLE ST BAKERSFIELD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31300.0,NaN,NaN,Tax Rate Area: 001-008Use Code: 101Zoning Clas...


In [ ]:
# Make new table/df with only desired columns
x = auctions[["Auction ID", "Unique #", "State", "County Name", "Parcel Location", "Minimum Bid Owed", "Property Description"]]
x[:5]

In [ ]:
# Remove 'BAKERSFIELD' from Address (later will need to make it 'CITY' for any CSV)

"""
- To iterate over rows, need to use method .iterrows(): 
        **_for label, row in df_name.iterrows()_** where label is the row label, and row is a Series
"""
# df['column_name'] = df['column_name'].str.replace('word_to_remove', '')


x['Parcel Location'] = x['Parcel Location'].str.replace('BAKERSFIELD', '')
x['Parcel Location'] = x['Parcel Location'].str.strip()
x.rename(columns={"Parcel Location": "Street Address"})
x.head()

In [ ]:
# TODO: Scrape sites for time deadline

# Combine Address w/ county, state, & country for geocoding to work
#x['Country'] = 'USA'
x.loc[:, 'Country'] = 'USA'
l_cols_concat = ['County Name','State','Country']
x['unique_address'] = x['Parcel Location'].str.cat(others=x[l_cols_concat], sep=', ',na_rep='')
x.head()

In [ ]:
# Geocoding with OpenStreetMap API (free service compared to Google Maps API)
import ssl
import certifi
import geopy.geocoders

ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
geolocator = geopy.geocoders.Nominatim(user_agent="gabriel.grinstein@gmail.com")

#address1_osm = x['unique_address'].iloc[0]
#location = geolocator.geocode(address1_osm)
#print('Latitude: '+str(location.latitude)+', Longitude: '+str(location.longitude))

In [ ]:
import numpy as np

def service_geocode(g_locator, address):
    """ Unique Address --> Lat/Lng (If location not found during geocoding, return NaN)"""
    location = g_locator.geocode(address)
    if location!=None:
      #return (location.latitude, location.longitude)
      return f"{location.latitude}, {location.longitude}"
    else:
      return np.NaN

In [ ]:
#TODO: ONLY use geocoding for desired locations (ie after trimming table); save rate limit & prevent timeouts

#x['LAT_LON'] = x['unique_address'].apply(lambda x: service_geocode(geolocator,x))
#x[['unique_address','LAT_LON']].head()

temp = x[:2]
temp['LAT_LON'] = temp['unique_address'].apply(lambda temp: service_geocode(geolocator, temp))


In [ ]:
temp

In [ ]:

# Put dataframe/table in sqlite3 DB (temp for now)
import sqlite3

# Path to your SQLite database file
db_file = 'auctions.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Put the DataFrame into the SQLite database
temp.to_sql('kern_ca', conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

"""
OR use SQLAlchemy:

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
temp.to_sql(name='kern_ca', con=engine)
"""

In [ ]:
# Get data from db and put back in a pandas dataframe

# Path to your SQLite database file
db_file = 'auctions.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# SQL query to select data from the database table
query = "SELECT * FROM kern_ca"

# Read data from the database into a Pandas DataFrame
temp2 = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Print the DataFrame to verify the data
print(temp2)

# Update the DataFrame as needed
# For example, you can add a new column or modify existing data
#temp2['new_column'] = 'new_value'

# Connect to the SQLite database again
conn = sqlite3.connect(db_file)

# Write the updated DataFrame back to the database
#df.to_sql('your_table_name', conn, if_exists='replace', index=False)

# Commit the changes and close the connection
#conn.commit()
conn.close()


In [ ]:
# Create UI Map using Flask, Jinja, & Mapbox GL JS
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def map_with_markers():
    # Path to your SQLite database file
    db_file = 'auctions.db'

    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    # SQL query to select data from the database table
    query = "SELECT * FROM kern_ca"

    # Read data from the database into a Pandas DataFrame
    temp3 = pd.read_sql_query(query, conn)

    # Close database connection
    conn.close()

    # Prepare data for Jinja template
    marker_data = []

    for index, row in temp3.iterrows():
        lat_lon = row['LAT_LON']
        coord_arr = lat_lon.split(',')
        lat = coord_arr[0]
        lon = coord_arr[1]
        
        price = row['Minimum Bid Owed']
        # Add additional information as needed

        # Format data for JavaScript
        marker_data.append({'lat': lat, 'lon': lon, 'price': price})

        # Render the template with marker data
        return render_template('map.html', marker_data=marker_data)

if __name__ == '__main__':
    app.run(debug=True)